# 00 Import Libraries

In [1]:
from openai import OpenAI
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
from time import sleep
import secrets
from flask import Flask, render_template, request, jsonify, session
from flask_session import Session

# 01 Functions

In [3]:
# FUNCTION TO OPEN FILES
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()
    
# FUNCTION TO CALCULATE SIMILARITY AND RETURN BEST MATCH
def similarity(question_embedding):
    
    # LOAD BOOKS FROM JSON FILE
    with open('json/books_info_embeddings.json', 'r') as f:
        products = json.load(f)
     
    # CALCULATE SIMILARITY BETWEEN QUESTION AND BOOKS
    best_match = None
    best_similarity = -1
    
    # LOOP THRU ALL BOOKS
    for product in products:

        product_embedding = np.array(product['text_embedding'])
        similarity = cosine_similarity([question_embedding], [product_embedding])[0][0]
        
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = product
    
    # PREPARE SYSTEM MESSAGE WITH DETAILS ABOUT BEST MATCHING BOOK
    best_match_system = f"""BOOK: {best_match['book_title']}
    CATEGORY: {best_match['book_category']}
    DESCRIPTION: {best_match['book_description']}
    RATING: {best_match['book_rating']}
    PRICE: {best_match['book_price']}
    AVAILABILITY: {best_match['book_availability']}"""
    
    return best_match_system

# FUNCTION TO CALL GPT CHATMODEL
def chatbot(messages, model="gpt-4-turbo-preview"):
    max_retry = 7
    retry = 0
    while True:
        try:
            response = client.chat.completions.create(model=model, messages=messages, temperature=0, max_tokens=1000, top_p=1, frequency_penalty=0, presence_penalty=0)
            return response
        except Exception as oops:
            print(f'\n\nError communicating with OpenAI: "{oops}"')
            if 'maximum context length' in str(oops):
                print('\n\n DEBUG: Trimming oldest message')
                continue
            retry += 1
            if retry >= max_retry:
                print(f"\n\nExiting due to excessive errors in API: {oops}")
                exit(1)
            print(f'\n\nRetrying in {2 ** (retry - 1) * 5} seconds...')
            sleep(2 ** (retry - 1) * 5)

# 02 Credentials

In [4]:
# LOAD OPENAI CREDENTIALS AND SYSTEM/USER MESSAGE
client = OpenAI(api_key=open_file('keys/openaiapikey.txt'))
system = open_file('prompts/system.txt')
user = open_file('prompts/user.txt')

# 03 Flask App

In [5]:
# FLASK APP
app = Flask(__name__)
secret_key = secrets.token_hex(16)
app.secret_key = secret_key
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

# APP
@app.route("/")
def index():

    if request.method == 'GET':
        session.clear()

    return render_template("index_V2.html")

@app.route('/get', methods=['POST'])
def get_bot_response():
    
    # USER INPUT EINLESEN
    user_input = request.json['message']
    
    # SESSION MESSAGES INITIALISIEREN
    if 'MESSAGES' not in session:
        session['MESSAGES'] = [{"role": "system", "content": system}]
        
        # RECOMMENDATIONS FÜR USER INPUT
        question_embedding = client.embeddings.create(input=user_input, model="text-embedding-3-small").data[0].embedding
        best_match_system = similarity(question_embedding)
        
        # RECOMMENDATION ANHÄNGEN
        session['MESSAGES'].append({"role": "user", "content": f"{user}{best_match_system}"})
        session['MESSAGES'].append({"role": "assistant", "content": "Ich lese"})
        
    # USER INPUT AN MESSAGES ANHÄNGEN
    session['MESSAGES'].append({"role": "user", "content": user_input})
        
    # GPT AUFRUFEN
    response = chatbot(session['MESSAGES'])
        
    # GPT RESPONSE AN CONVERSATION ANHÄNGEN
    last_bot_message = response.choices[0].message.content
    session['MESSAGES'].append({'role': 'assistant', 'content': last_bot_message})
    
    # SESSION MESSAGES SPEICHERN
    session.modified = True
    
    # Response als JSON zurückgeben
    return jsonify(message=last_bot_message)

# 04 Run the App

In [7]:
if __name__ == "__main__":
    port = 8083
    app.run(host='0.0.0.0', port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8083
 * Running on http://192.168.178.153:8083
Press CTRL+C to quit
127.0.0.1 - - [10/Apr/2024 10:25:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2024 10:25:06] "GET /static/css/style_gpt.css HTTP/1.1" 304 -
127.0.0.1 - - [10/Apr/2024 10:25:06] "GET /static/images/start-logo.png HTTP/1.1" 304 -
127.0.0.1 - - [10/Apr/2024 10:25:06] "GET /static/images/br-logo.png HTTP/1.1" 304 -
127.0.0.1 - - [10/Apr/2024 10:25:06] "GET /static/js/chat.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Apr/2024 10:25:06] "GET /static/fonts/Korb-Bold.woff2 HTTP/1.1" 304 -
